In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
import sklearn
import warnings 
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df = pd.read_csv("data/train.csv")
df = df.sample(1200, random_state=786)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().any()

In [ ]:
df = df.rename(columns={'blue':'bluetooth', 'fc':'front_cam_mp', 'sc_h':'screen_ht', 'sc_w':'screen_wt'})

In [ ]:
df = df.rename(columns={'pc':'back_cam_mp'})

In [ ]:
df = df.drop(columns=['m_dep', 'mobile_wt', 'px_height', 'px_width'])

In [ ]:
categorical_features = ['bluetooth', 'clock_speed', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi']
continuous_features = ['battery_power', 'front_cam_mp', 'int_memory', 'n_cores', 'back_cam_mp', 'ram']
TARGET = ['price_range']

In [ ]:
df[continuous_features].describe()

In [ ]:
df.shape

In [ ]:
df['battery_power'].describe()

In [ ]:
df['bluetooth'].unique()

In [ ]:
df['clock_speed'].unique()

In [ ]:
df['dual_sim'].value_counts()

In [ ]:
df['front_cam_mp'].unique()

In [ ]:
df['four_g'].value_counts()

In [ ]:
df['int_memory'].describe()

In [ ]:
df['n_cores'].value_counts()

In [ ]:
df['back_cam_mp'].describe()

In [ ]:
df['ram'].describe()

In [ ]:
df['screen_ht'].unique()

In [ ]:
df['screen_wt'].unique()

In [ ]:
df['talk_time'].describe()

In [ ]:
df['three_g'].value_counts()

In [ ]:
df['touch_screen'].value_counts()

In [ ]:
df['four_g'].value_counts()

In [ ]:
df['wifi'].value_counts()

In [ ]:
df['price_range'].value_counts()

In [ ]:
df['clock_speed'] = pd.cut(df['clock_speed'], bins=[0, 1, 2, 3], labels = ['low', 'mid', 'high'])


In [ ]:
df.head()

In [ ]:
level_map = {'low':0, 'mid':1, 'high':2}
df['clock_speed'] = df['clock_speed'].replace(level_map)

In [ ]:
df.head()

In [ ]:
df[['screen_ht', 'screen_wt']].describe()

In [ ]:
df['screen_ht'] = df['screen_ht'].astype(float)
df['screen_wt'] = df['screen_wt'].astype(float)

In [ ]:
x = df[df['screen_wt']==0]['screen_ht'].value_counts().index.tolist()

In [ ]:
arr = []
for d in df.loc[df['screen_wt']==0]['screen_ht']:
    arr.append(d)

set(arr)

In [ ]:
mean_width = {}
for d in set(arr):
    total = 0
    n = 0
    for width in df.loc[df['screen_ht'] == d]['screen_wt']:
        if width == 0:
            pass
        total += width
        n += 1
        mean = round(total/n, 2)
    print("Mean width for height", d, "=", mean)
    mean_width[d] = mean

In [ ]:
for z in x:
    df['screen_wt'] = np.where(((df['screen_wt']==0.0) & (df['screen_ht']==z)), mean_width.get(z), df['screen_wt'])

In [ ]:
df.head()

In [ ]:
df['screen_size'] = df['screen_ht']**2 + df['screen_wt']**2
df['screen_size'] = np.sqrt(df['screen_size'])
df['screen_size'] = df['screen_size']/2.54
df['screen_size'] = df['screen_size'].round(2)
df.drop(columns=['screen_ht', 'screen_wt'], inplace=True)

In [ ]:
p = pd.DataFrame(df['price_range'])
df.drop(columns=TARGET, inplace=True)
df = df.join(p)

In [ ]:
df.head()

# Data Visualisation
## 1 variable 

In [ ]:
# Bar Plot
fig = plt.figure(figsize = (6, 4))
title = fig.suptitle("No. of Cores vs.Frequency", fontsize = 14)
fig.subplots_adjust(top=0.85, wspace=0.3)

ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel("No.of Cores")
ax.set_ylabel("Frequency") 
w_q = df['n_cores'].value_counts()
w_q = (list(w_q.index), list(w_q.values))
ax.tick_params(axis='both', which='major', labelsize=8.5)
bar = ax.bar(w_q[0], w_q[1], color='steelblue', 
        edgecolor='black', linewidth=1)

In [ ]:
labels = ['low', 'mid', 'high']
df['clock_speed'].value_counts().plot(kind='pie', autopct='%.2f')
plt.tight_layout()
plt.legend(labels)
plt.show()

In [ ]:
df['screen_size'].plot(kind='density')

## 2 variable 

In [ ]:
plt.hist2d(df['price_range'], df['ram'], bins=(4, 16), cmap='Blues')
cb = plt.colorbar()
cb.set_label('counts in bin')

In [ ]:
sns.set(style="ticks", color_codes=True)

In [ ]:

# Using subplots or facets along with Bar Plots
fig = plt.figure(figsize = (10, 4))
title = fig.suptitle("Dual_sim vs. talk_time", fontsize=14)
fig.subplots_adjust(top=0.85, wspace=0.3)

# Non Dual Sim
ax1 = fig.add_subplot(1,2, 1)
ax1.set_title("Non Dual Sim")
ax1.set_xlabel("Talk-Time")
ax1.set_ylabel("Frequency") 
rw_q = df[df['dual_sim'] == 0]['talk_time'].value_counts()
rw_q = (list(rw_q.index), list(rw_q.values))
ax1.set_ylim([0,70])
ax1.tick_params(axis='both', which='major', labelsize=8.5)
bar1 = ax1.bar(rw_q[0], rw_q[1], color='red', 
               edgecolor='black', linewidth=1)

# Dual Simw
ax2 = fig.add_subplot(1,2, 2)
ax2.set_title("Dual Sim")
ax2.set_xlabel("Talk-time")
ax2.set_ylabel("Frequency") 
ww_q = df[df['dual_sim'] == 1]['talk_time'].value_counts()
ww_q = (list(ww_q.index), list(ww_q.values))
ax2.set_ylim([0, 70])
ax2.tick_params(axis='both', which='major', labelsize=8.5)
bar2 = ax2.bar(ww_q[0], ww_q[1], color='white', 
               edgecolor='black', linewidth=1)

In [ ]:
df.boxplot(column='battery_power', by='price_range')
plt.show()

## 3 variable 

In [ ]:

# Scaling attribute values to avoid few outiers
cols = ['ram', 'int_memory', 'screen_size', 'battery_power','price_range']
pp = sns.pairplot(df[cols], hue='price_range', size=1.8, aspect=1.8, 
                  palette={0: "#FF9999", 1: "#FFE888", 2:"#2A9D8F", 3:"#E63946"},
                  plot_kws=dict(edgecolor="black", linewidth=0.5))
fig = pp.fig 
fig.subplots_adjust(top=0.93, wspace=0.3)
t = fig.suptitle('Wine Attributes Pairwise Plots', fontsize=14)

In [ ]:
sns.relplot(y="int_memory", x="front_cam_mp", hue='four_g', kind="line", data=df)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt



fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(df['battery_power'], df['back_cam_mp'], df['screen_size'], c='r', marker='o')

ax.set_xlabel('Battery Power')
ax.set_ylabel('Back_Camera_Px')
ax.set_zlabel('Screen_Size')

plt.show()

# Methodology

We are using classification task for this machine learning project. The 4 algorithms used to predit the models are:<br><br>
1. K-Nearest Neigbors (commonly known as KNN)<br>
2. Decision Tree<br>
3. Random Forest<br>
4. Support Vector Machine (also known as SVM)<br><br>

Firstly,we have applied cross fold validation on the entire feature present in the dataset after pre-processing & <br>found CV score as **'0.38'**. As the score seemed to be lower,we then decided to go for feature selection to<br> see if any improvement in the accuracy.<br> We applied 'f-score' & 'random forest importance' methods for feature selection and compared the accuracy of both.<br> Finally we ended up with f-score as best features estimator and used it for further analysis.<br><br>
We then applied the above mentioned algorithms on the best features given by f-score method and estimated the<br> accuracy of all models. Also for each algorithm, we tuned the parameters and visualised to get the best accuracy<br> score for corresponding model.<br>
Lastly we evaluated the algorithms using the performance metrics and performance comparison using paired t-test






# Feature Selection
Feature selection is the process where you automatically select features which contribute most to your prediction<br> variable. Sometimes having many features can decrease the accuracy of model.<br><br>

1. Performance with full sets of features:<br><br>
We first accessed the performance using all the features of our data. We used Stratified-K-fold methods with <br>splits = '5' and repetitions ='3' with scoring metric set to accuracy & lastly computed the result using <br>cross_val_score () .



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score,RepeatedStratifiedKFold

Data= df.drop(columns=['price_range'])
target = df[TARGET]
Data = preprocessing.MinMaxScaler().fit_transform(Data)

clf = KNeighborsClassifier(n_neighbors=1)
cv_method = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=786)
scoring_metric = 'accuracy'
cv_results_full = cross_val_score(estimator=clf, X=Data, y=target, cv=cv_method,scoring=scoring_metric)

cv_results_full.mean().round(2)

With full set of features & 1 neigbor classifier, we achieved the accuracy score of **38%**.

2. Feature selection using f-score:<br><br>
F-score method selects the features based on relationship between descriptive feature and target feature using<br>F-distribution. We now set number of features to 8. The fs_indices_fscore returns us top 8 features sorted<br>highest to lowest. 
 

In [ ]:
Data = df.drop(columns=['price_range'])
target = df[TARGET]
Data = preprocessing.MinMaxScaler().fit_transform(Data)

In [ ]:
from sklearn import feature_selection as fs
num_features = 8
fs_fit_fscore = fs.SelectKBest(fs.f_classif, k=num_features)
fs_fit_fscore.fit_transform(Data, target)
fs_indices_fscore = np.argsort(np.nan_to_num(fs_fit_fscore.scores_))[::-1][0:num_features]
fs_indices_fscore

In [ ]:
best_features_fscore = df.columns[fs_indices_fscore].values
best_features_fscore

* We got *'ram'*, *'battery_power'*, *'int_memory'*, *'clock_speed'*, *'screen_size'*, *'n_cores'*, *'talk_time'*and <br>*'front_cam_mp'* as best features based on F-score.

In [ ]:
feature_importances_fscore = fs_fit_fscore.scores_[fs_indices_fscore]
feature_importances_fscore

In [ ]:
import altair as alt

def plot_imp(best_features, scores, method_name, color):
    
    df = pd.DataFrame({'features': best_features, 
                       'importances': scores})
    
    chart = alt.Chart(df, 
                      width=500, 
                      title=method_name + ' Feature Importances'
                     ).mark_bar(opacity=0.75, 
                                color=color).encode(
        alt.X('features', title='Feature', sort=None, axis=alt.AxisConfig(labelAngle=45)),
        alt.Y('importances', title='Importance')
    )
    
    return chart

* Plotting the best_features_fscores to visualised the feature importance.

In [ ]:
plot_imp(best_features_fscore, feature_importances_fscore, 'F-Score', 'red')


*Accessing the performance of the selected features using cross validation. *

In [ ]:
cv_results_fscore = cross_val_score(estimator=clf,
                             X=Data[:, fs_indices_fscore],
                             y=target, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_results_fscore.mean().round(3)

3. Feature selection using Random Forest Importance<br><br>
Random Forest importance (RFI) is widely used feature selector because of the accuracy, robustness and <br>ease of use it gives. It tells us about how much accuracy is decreased when a variable is excluded <br>and decrease in gini impurity when a variable is chosen to split node.


In [ ]:
Data= df.drop(columns=['price_range'])
target=df[TARGET]
Data=preprocessing.MinMaxScaler().fit_transform(Data)

In [ ]:
Data

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rfi = RandomForestClassifier(n_estimators=100)
model_rfi.fit(Data, target)
fs_indices_rfi = np.argsort(model_rfi.feature_importances_)[::-1][0:num_features]

In [ ]:
best_features_rfi = df.columns[fs_indices_rfi].values
best_features_rfi

* We got *'ram'*, *'battery_power'*, *'screen_size'*,*'int_memory'*, *'talk_time'*, *'front_cam_mp'*,<br>*'back_cam_mp'* and *'n_cores'*,as best features based on random forest importance.

In [ ]:
feature_importances_rfi = model_rfi.feature_importances_[fs_indices_rfi]
feature_importances_rfi

* Plotting the best_features_rfi to visualised the feature importance

In [ ]:
plot_imp(best_features_rfi, feature_importances_rfi, 'Random Forest', 'green')

*Accessing the performance of the selected features using cross validation.*

In [ ]:
cv_results_rfi = cross_val_score(estimator=clf,
                             X=Data[:, fs_indices_rfi],
                             y=target, 
                             cv=cv_method, 
                             scoring=scoring_metric)
cv_results_rfi.mean().round(3)

Finding the overall performance:<br>
We found that F-score feature selector gives us good accuracy score as compared to random forest importance.<br> Hence we choose '{best_feature_f-score}' for further fitting the model.


In [ ]:
print('Full Set of Features:', cv_results_full.mean().round(3))
print('F-Score:', cv_results_fscore.mean().round(3))
print('RFI:', cv_results_rfi.mean().round(3))

### Splitting the data into training and test set

We have selected the subset(1200) of our entire data i.e(2000) for model fitting and evaluation.<br>We have split the data into 70 :30 ratio . i.e  70% of our data to build a model and 30% data to test it to ensure<br> that we measure the accuracy based on unseen data.

In [ ]:
Data = df[best_features_fscore].copy()
target = df[TARGET]
Data = preprocessing.MinMaxScaler().fit_transform(Data)

In [ ]:
from sklearn.model_selection import train_test_split

D_train, D_test, t_train, t_test = train_test_split(Data, target, test_size=0.3, random_state=786)

# Model Fitting

## 1.K-Nearest Neighbor (KNN)<br>
We fit a ‘KNeighborClassifier’ with default parameter values as ‘n_neigbors =5’ and ‘P=2’. n_neigbors value is <br>the number of neigbors to be used and P=2 is the Euclidean distance metric .<br>
The score function returns the accuracy of classifier on the test data. Accuracy is ratio of total correctly<br> predicted observations upon total number of observations.  Computed accuracy found was 59.44%<br>


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier(n_neighbors=5, p=2)
knn_classifier.fit(D_train, t_train) 
knn_classifier.score(D_test, t_test)

## Hyperparameter tuning using Grid Search
Grid-search is used to find the optimal hyperparameters of a model which results in the most ‘accurate’ predictions.<br>

Below we have defined a function for *'grid search'* to which we pass the classifier (KNN, DT, RF, and SVM) and<br> training data.
* We have defined different parameters for each algorithm in the ‘grid_params’ method. 
* The function returns us best model parameters and model score based on the parameters given.
* In addition we include repeated stratified cv method.
* Also we tell sklean library which metric to optimize i.e. accuracy in our case.<br>

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

def grid_search(D_train, t_train, clf):
   
    if isinstance(clf, KNeighborsClassifier): 
        grid_params = {
        'n_neighbors':[3, 5, 7, 9, 11, 13, 15],
        'p':[1, 2, 3]
        }
    elif isinstance(clf, DecisionTreeClassifier): 
        grid_params = {
        'criterion':['gini','entropy'],
        'min_samples_split':[2, 3, 4],
        'max_depth':[1, 2, 3, 4, 5, 6, 7, 8]
        }
    elif isinstance(clf, RandomForestClassifier):
        grid_params = {
        'n_estimators':[110, 130, 150, 200],
        'criterion':['gini','entropy'],
        'min_samples_split':[2, 3, 4],
        'max_depth':[3, 4, 5]
        
        }
    elif isinstance(clf, SVC):
       grid_params = {
            'C':[1, 10, 50, 100],
            'gamma':[1, 0.1, 0.05, 0.001],
            'kernel':['rbf', 'poly', 'sigmoid']
        }
    else : 
        raise ValueError("unkown classifier")

    gs = GridSearchCV(
        estimator = clf,
        param_grid = grid_params,
        verbose = 3,
        cv = cv_method,
        n_jobs = -1,
        refit = True   
    )

    gs_results = gs.fit(D_train, t_train)
    p = gs_results.best_params_
    model = gs_results.best_estimator_
    return model, p, gs_results

* With n_neigbors: [3, 5, 7, 9, 11, 13, 15] and P: [1, 2, 3] the grid search function finds out the best parameter <br>values and calculates the model score.

In [ ]:
knn_model, knn_best_estimate, knn_result = grid_search(D_train, t_train, knn_classifier)


In [ ]:
knn_best_estimate

In [ ]:
knn_model.score(D_test, t_test)

* KNN classifier with n_neighbor =15 and p=1 predicted the model mean score of 68.8% 

In [ ]:
results_KNN = pd.DataFrame(knn_result.cv_results_['params'])
results_KNN['test_score'] = knn_result.cv_results_['mean_test_score']
results_KNN.head()


In [ ]:
results_KNN['metric'] = results_KNN['p'].replace([1, 2, 3], ["Manhattan", "Euclidean", "Minkowski"])
results_KNN.head()

### Plotting the KNN Performance comparison. 
We know visualise the hyper parameter tuning results from cross fold validation. We plot using altair module.<br> The plot shows that at all values of K with Manhattan distance (p=1) outperforms others.

In [ ]:
import altair as alt

alt.Chart(results_KNN, 
          title='KNN Performance Comparison'
         ).mark_line(point=True).encode(
    alt.X('n_neighbors', title='Number of Neighbors'),
    alt.Y('test_score', title='Mean CV Score', scale=alt.Scale(zero=False)),
    color='metric'
)

#### Advantages of KNN Classifier:
*   The algorithm is simple and easy to implement .
*   The algorithm is versatile and can be used for classification , regression and search.<br><br>

#### Disadvantages:
*   The algorithm gets slower as number of independent variables increases where predictions needs to be made<br> rapidly.

#### Limitations:
*   Need to have high computing resources to speedly handle the data.

## 2.Decisoin Tree Clasification

Decision trees are non-parametric supervised learning methods used for classification. The main aim of this is to <br> define a model that gives value of target feature by learning decision rule inferred from data features.<br><br>
Fitting the decision tree classifier with default values and random state = 786 which was selected at the very <br>beginning.<br><br>
The score function returns the accuracy of classifier on the test data. Accuracy is ratio of total correctly <br>predicted observations upon total number of observations.<br> The accuracy measured was 76.38%.<br>


In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=786)
dt_classifier.fit(D_train, t_train)
dt_classifier.score(D_test, t_test)

* Out of Criterion = [‘gini’ ,entropy’] , min_sample_split=[2, 3, 4] & max_depth=[1, 2, 3, 4, 5, 6, 7, 8] the grid <br> search function finds out the best parameter values and calculates the model score.

In [ ]:
dt_model, dt_best_estimate, dt_result = grid_search(D_train, t_train, dt_classifier)

In [ ]:
dt_best_estimate

In [ ]:
dt_model.score(D_test, t_test)

*With Criterion ‘gini’ , max_depth =4 and min_sample_splits of 2 the model predicts the accuracy 76.6%*

In [ ]:
results_DT = pd.DataFrame(dt_result.cv_results_['params'])
results_DT['test_score'] = dt_result.cv_results_['mean_test_score']
results_DT.head()

### Plotting the DT Performance Comparison 
Also from the plot we visualise the best hyperparamters as ‘gini’ and ‘max_depth:4’


In [ ]:
alt.Chart(results_DT, 
          title='DT Performance Comparison'
         ).mark_line(point=True).encode(
    alt.X('max_depth', title='Maximum Depth'),
    alt.Y('test_score', title='Mean CV Score', aggregate='average', scale=alt.Scale(zero=False)),
    color='criterion'
)


#### Advantages of Decision tree classifier
*	Inexpensive to construct. 
*	Easy to interpret for small size trees. 
*	Fast at classifying unknown records.

#### Disadvantages
*	Decision tree models are often biased towards splits on features.
*	Large trees can be difficult to interpret.
*	Small change in training data can account for large change to decision logic.


## 3.Random Forest Classifier
A random forest is a Meta estimator that fits number of decision tree classifier on various sub-samples<br> and uses mean to advance the accuracy and avoid over-fitting.<br><br>
Fitting the random forest classifier with default estimator n= 100 i.e. number of trees in the forest,<br> criterion ‘gini and max_depth 2.<br>

The score function returns the accuracy of classifier on the test data. Accuracy is ratio of total correctly predicted observations upon total number of observations. The accuracy measured was *73.6%*.


In [ ]:
rf_classifier = RandomForestClassifier(random_state=786,n_estimators=100,max_depth=2,criterion='gini')
rf_classifier.fit(D_train, t_train)
rf_classifier.score(D_test, t_test)

* Out of the given parameters given to grid search function criterion =[‘gini’, ‘entropy’], <br>n_estimators= [110, 130, 150, 200], max_depth=[3, 4, 5] and min_sample_split= [2, 3, 4] it calculates & returns best <br>parameters with model score.

In [ ]:
rf_model, rf_best_estimate, rf_result = grid_search(D_train, t_train, rf_classifier)

In [ ]:
rf_best_estimate

In [ ]:
rf_model.score(D_test, t_test)

* The model predicts the accuracy score of 76.9%.*

In [ ]:
results_RF = pd.DataFrame(rf_result.cv_results_['params'])
results_RF['test_score'] = rf_result.cv_results_['mean_test_score']
results_RF.head()

### Plotting the RF Performance Comparison 
From the plot we visualise that at max_depth =4 , gini overpowers entropy . 


In [ ]:
alt.Chart(results_RF, 
          title='RF Performance Comparison'
         ).mark_line(point=True).encode(
    alt.X('max_depth', title='Maximum Depth'),
    alt.Y('test_score', title='Mean CV Score', aggregate='average', scale=alt.Scale(zero=False)),
    color='criterion'


#### Advantages of Random forest classifier
*	No need of any feature selection 
*	Easier to make parallel models 
*	If larger parts of features are lost , accuracy can still be maintained.

#### Disadvantages
*	Fits for some noisy data 
*	Time complexity- much harder and time consuming to construct.

#### Limitations
*	Heavy computation resources.


## 4.Support Vector Machine classifier 
SVM is linear model for classification problem. The idea of SVM is simple. The algorithm creates <br>a line or hyperplane which separates the data into classes.<br><br>
We fit the model with default kernel as rbf and regularisation value=1.0 parameters .<br><br>
The score function returns the accuracy of classifier on the test data. Accuracy is ratio of total<br> correctly predicted observations upon total number of observations. The accuracy measured was 78.6%.


In [ ]:
svm_classifier = SVC()
svm_classifier.fit(D_train, t_train)
svm_classifier.score(D_test, t_test)

* The parameters passed to grid search function were gamma values=[0.1, 0.05, 0.001, 1] as the value <br> must be between 0.1 to 1. Kernels=[‘rbf’, ‘poly’, ‘sigmoid’] with C=[1, 10, 50, 100].

In [ ]:
svm_model, svm_best_estimate, svm_result = grid_search(D_train, t_train, svm_classifier)

In [ ]:
svm_best_estimate

In [ ]:
svm_model.score(D_train, t_train)

*The model predicts the accuracy score of 83.4% with best parameters .*

In [ ]:
results_SVM = pd.DataFrame(svm_result.cv_results_['params'])
results_SVM['test_score'] = svm_result.cv_results_['mean_test_score']
results_SVM.head()

### Plotting the SVM Performance Comparison 
From the plot we visualise that at max_depth =4 , gini overpowers entropy . 


In [ ]:
alt.Chart(results_SVM, 
          title='SVM Performance Comparison'
         ).mark_line(point=True).encode(
    alt.X('C', title='Regularisation Parameter'),
    alt.Y('test_score', title='Mean CV Score', aggregate='average', scale=alt.Scale(zero=False)),
    color='kernel'
)

#### Advantages of Support Vector Machine
*	Work well when there is clean margin of separation.
*	Memory efficient
#### Disadvantages
*	Not suitable for larger data sets
*	SVM does not perform well when data set has more noise or target class is overlapping.


## Performance Comparison
After testing the classifier  by considering the train data  and using it in cross validation way, <br>we know perform the paired t-test in order to understand if the difference between performance is statistically<br> significant for any 2 classifiers.<br><br>
Firstly we calculate the cross_val_score  and then compare it with all models as:
*	KNN-DT
*	KNN-RF
*	KNN-SVM
*	DT-RF
*	DT-SVM
*	RF_SVM<br>

From scipy library we import the stats module to run the t-test .


In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

cv_method_ttest = StratifiedKFold(n_splits=10, random_state=786)
cv_results_KNN = cross_val_score(estimator=knn_model, X=Data, y=target, cv=cv_method_ttest, n_jobs=-1, scoring='accuracy')              

In [ ]:
cv_results_KNN.mean()

In [ ]:
cv_results_RF = cross_val_score(estimator=rf_model, X=Data, y=target, cv=cv_method_ttest, n_jobs=-1, scoring='accuracy')
cv_results_RF.mean()

In [ ]:
cv_results_DT = cross_val_score(estimator=dt_model, X=Data, y=target, cv=cv_method_ttest, n_jobs=-1, scoring='accuracy')
cv_results_DT.mean()

In [ ]:
cv_results_SVM = cross_val_score(estimator=svm_model, X=Data, y=target, cv=cv_method_ttest, n_jobs=-1, scoring='accuracy')
cv_results_SVM.mean()

In [ ]:
from scipy import stats

print(stats.ttest_rel(cv_results_KNN, cv_results_DT))
print(stats.ttest_rel(cv_results_KNN, cv_results_RF))
print(stats.ttest_rel(cv_results_KNN, cv_results_SVM))

print(stats.ttest_rel(cv_results_DT, cv_results_RF))
print(stats.ttest_rel(cv_results_DT, cv_results_SVM))

print(stats.ttest_rel(cv_results_RF, cv_results_SVM))

*The Pair KNN-SVM gives statistically significant value of 0.0002 which is less than 0.05.*

# Model Evaluation 
Model evaluation is one of the important step required to determine the best model, how well the model will perform.<br><br>
The target variable for our dataset was multinomial. That is target feature is categorical with 4 different <br>level {0, 1, 2, 3}. It refers to different price range ={‘low’, ‘mid’, ‘high’, ‘v high’}. <br>Hence we cannot use binary metric such as roc_auc curve to evaluate multinomial classifier.<br><br>
Below are the evaluation metrics used to find the accuracy, classification report and average model accuracy<br> for each model.


In [ ]:
from sklearn import metrics
def print_model_stats(model, D_test, t_test):
    pred = model.predict(D_test)
    print("=========={model_name} Model Statistics=============".format(model_name=model.__class__.__name__))
    print("Accuracy score:", metrics.accuracy_score(t_test, pred))
    print("Confusion Matrix:\n", metrics.confusion_matrix(t_test, pred))
    print("Classification report:\n", metrics.classification_report(t_test, pred))
    print("Average model accuracy:", metrics.balanced_accuracy_score(t_test, pred))

In [ ]:
print_model_stats(knn_model, D_test, t_test)

In [ ]:
print_model_stats(dt_model, D_test, t_test)

In [ ]:
print_model_stats(rf_model, D_test, t_test)

In [ ]:
print_model_stats(svm_model, D_test, t_test)

*Hence we conclude that SVM gives us the best model accuracy and should be used for this case study.*